In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import psycopg2
from PIL import Image, ImageTk
import os
import webbrowser
import pandas as pd
from tkcalendar import DateEntry
from datetime import datetime
import pyperclip



class ResumeViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Просмотр резюме")
        self.root.geometry("1000x700")
        self.root.minsize(800, 600)
        
        # Конфигурация базы данных
        self.db_config = {
            'host': 'localhost',
            'port': '5433',
            'database': 'hh_test',
            'user': 'postgres',
            'password': '123'
        }
        
        # Проверяем существование колонки comment_browser
        self.comment_browser_exists = self.check_comment_browser_column()
        
        # Путь к папке с фотографиями
        self.photos_base_path = os.path.join(os.environ['USERPROFILE'], 'Desktop')
        
        # Текущий индекс просматриваемого резюме
        self.current_index = 0
        self.filter_mode = False
        self.current_filter = ""
        self.image_cache = {}
        self.current_display_data = pd.DataFrame()
        self.session_min = 0
        self.session_max = 0
        self.grid_view_mode = False
        self.grid_page_size = 9
        
        # Размеры изображений
        self.single_image_size = (400, 400)
        self.grid_image_size = (200, 200)
        
        # Проверяем существование таблицы шаблонов
        self.templates_exist = self.check_templates_table()
        
        # Загружаем данные
        self.load_data()
        
        # Создаем интерфейс
        self.create_widgets()
        
        # Показываем первое резюме
        self.show_resume()

        # Загружаем белую фотографию
        self.blank_photo_path = os.path.join(self.photos_base_path, "белое.jpg")
        self.blank_photo = None
        if os.path.exists(self.blank_photo_path):
            try:
                img = Image.open(self.blank_photo_path)
                img = img.resize(self.single_image_size, Image.LANCZOS)
                self.blank_photo = ImageTk.PhotoImage(img)
                
                img_grid = img.resize(self.grid_image_size, Image.LANCZOS)
                self.blank_photo_grid = ImageTk.PhotoImage(img_grid)
            except Exception as e:
                print(f"Ошибка загрузки белого фото: {e}")
        
        # Биндим клавиши
        self.root.bind('<Key>', self.handle_key_press)

    def check_comment_browser_column(self):
        """Проверяет существование колонки comment_browser в таблице info_res"""
        try:
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        SELECT column_name 
                        FROM information_schema.columns 
                        WHERE table_name='info_res' AND column_name='comment_browser'
                    """)
                    return cursor.fetchone() is not None
        except Exception as e:
            print(f"Ошибка при проверке колонки comment_browser: {e}")
            return False
        
    def check_templates_table(self):
        """Проверяет существование таблицы шаблонов комментариев"""
        try:
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        SELECT EXISTS (
                            SELECT FROM information_schema.tables 
                            WHERE table_name = 'comment_templates'
                        )
                    """)
                    return cursor.fetchone()[0]
        except Exception as e:
            print(f"Ошибка при проверке таблицы шаблонов: {e}")
            return False
    
    def load_templates(self):
        """Загружает шаблоны комментариев из базы данных"""
        if not self.templates_exist:
            return []
        
        try:
            with psycopg2.connect(**self.db_config) as conn:
                return pd.read_sql("SELECT id, template_text FROM comment_templates ORDER BY id", conn)
        except Exception as e:
            print(f"Ошибка загрузки шаблонов: {e}")
            return pd.DataFrame()
    
    def save_template(self, template_text):
        """Сохраняет новый шаблон в базу данных"""
        if not template_text.strip():
            messagebox.showwarning("Предупреждение", "Текст шаблона не может быть пустым")
            return False
        
        try:
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        INSERT INTO comment_templates (template_text)
                        VALUES (%s)
                        RETURNING id
                    """, (template_text.strip(),))
                    conn.commit()
                    return True
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось сохранить шаблон: {e}")
            return False
    
    def load_data(self):
        """Загружает данные из базы данных"""
        try:
            with psycopg2.connect(**self.db_config) as conn:
                # Получаем данные о резюме с информацией о сессии
                query = """
                    SELECT i.*, p.html_file_name, p.parsing_comment as session_comment
                    FROM info_res i
                    LEFT JOIN parsing_metadata p ON i.session_id = p.session_id
                    ORDER BY i.id
                """
                self.resumes = pd.read_sql(query, conn)
                
                if 'visit_time' in self.resumes.columns and self.resumes['visit_time'].dtype == object:
                    self.resumes['visit_time'] = pd.to_datetime(self.resumes['visit_time'], errors='coerce')
                
                session_range = pd.read_sql("""
                    SELECT MIN(session_id) as min_session, MAX(session_id) as max_session 
                    FROM info_res
                """, conn)
                
                self.session_min = session_range.iloc[0]['min_session'] or 0
                self.session_max = session_range.iloc[0]['max_session'] or 0
                
                date_range = pd.read_sql("""
                    SELECT MIN(visit_time) as min_date, MAX(visit_time) as max_date 
                    FROM info_res
                """, conn)
                
                self.min_date = date_range.iloc[0]['min_date'] or datetime.now().date()
                self.max_date = date_range.iloc[0]['max_date'] or datetime.now().date()
                
                self.current_display_data = self.resumes.copy()
                
                if self.templates_exist:
                    self.templates = self.load_templates()
                
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось загрузить данные: {e}")
            self.resumes = pd.DataFrame()
            self.current_display_data = pd.DataFrame()
    
    def create_widgets(self):
        """Создает элементы интерфейса"""
        # Основной контейнер
        self.main_container = ttk.Frame(self.root)
        self.main_container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Фрейм для фильтров
        self.filter_frame = ttk.Frame(self.main_container)
        self.filter_frame.pack(fill=tk.X, pady=5)
        
        # Фрейм для комментария из браузера (НОВОЕ ПОЛЕ)
        self.browser_comment_frame = ttk.Frame(self.main_container)
        self.browser_comment_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(self.browser_comment_frame, text="Комментарий из браузера (R):").pack(side=tk.LEFT, padx=5)
        self.browser_comment_entry = ttk.Entry(self.browser_comment_frame, width=50)
        self.browser_comment_entry.pack(side=tk.LEFT, padx=5, fill=tk.X, expand=True)
        
        # Кнопка сохранения комментария из браузера
        self.save_browser_comment_btn = ttk.Button(self.browser_comment_frame, text="Сохранить",
                                                 command=self.save_browser_comment)
        self.save_browser_comment_btn.pack(side=tk.LEFT, padx=5)
        
        # Фрейм для массового добавления комментариев
        self.bulk_comment_frame = ttk.Frame(self.main_container)
        self.bulk_comment_frame.pack(fill=tk.X, pady=5)

        # Поле для фильтрации по комментариям
        ttk.Label(self.filter_frame, text="Фильтр по комментариям:").pack(side=tk.LEFT, padx=5)
        self.filter_entry = ttk.Entry(self.filter_frame, width=20)
        self.filter_entry.pack(side=tk.LEFT, padx=5)
        
        # Кнопка выбора шаблона комментария
        if self.templates_exist:
            self.template_btn = ttk.Button(self.filter_frame, text="Выбрать шаблон", 
                                         command=self.show_template_dropdown)
            self.template_btn.pack(side=tk.LEFT, padx=5)
            
            self.add_template_btn = ttk.Button(self.filter_frame, text="+ Шаблон",
                                             command=self.add_template_dialog)
            self.add_template_btn.pack(side=tk.LEFT, padx=5)
        
        # Фильтр по номеру сессии
        ttk.Label(self.filter_frame, text="С сессии:").pack(side=tk.LEFT, padx=5)
        self.session_from = ttk.Entry(self.filter_frame, width=5)
        self.session_from.pack(side=tk.LEFT, padx=5)
        self.session_from.insert(0, str(self.session_min))
        
        ttk.Label(self.filter_frame, text="по:").pack(side=tk.LEFT, padx=0)
        self.session_to = ttk.Entry(self.filter_frame, width=5)
        self.session_to.pack(side=tk.LEFT, padx=5)
        self.session_to.insert(0, str(self.session_max))
        
        # Фильтр по дате посещения
        ttk.Label(self.filter_frame, text="Дата посещения с:").pack(side=tk.LEFT, padx=5)
        self.date_from = DateEntry(self.filter_frame, width=12, background='darkblue',
                                 foreground='white', borderwidth=2, date_pattern='dd.mm.yyyy',
                                 mindate=self.min_date, maxdate=self.max_date)
        self.date_from.pack(side=tk.LEFT, padx=5)
        
        ttk.Label(self.filter_frame, text="по:").pack(side=tk.LEFT, padx=0)
        self.date_to = DateEntry(self.filter_frame, width=12, background='darkblue',
                               foreground='white', borderwidth=2, date_pattern='dd.mm.yyyy',
                               mindate=self.min_date, maxdate=self.max_date)
        self.date_to.pack(side=tk.LEFT, padx=5)
        
        # Кнопка применения фильтра
        self.filter_btn = ttk.Button(self.filter_frame, text="Применить фильтр", 
                                   command=self.apply_filter)
        self.filter_btn.pack(side=tk.LEFT, padx=5)
        
        # Кнопка сброса фильтра
        self.reset_filter_btn = ttk.Button(self.filter_frame, text="Сбросить фильтр",
                                         command=self.reset_filter)
        self.reset_filter_btn.pack(side=tk.LEFT, padx=5)
        
        # Checkbox для переключения режима просмотра
        self.grid_mode_var = tk.BooleanVar()
        self.grid_mode_check = ttk.Checkbutton(
            self.filter_frame, 
            text="Режим просмотра сеткой", 
            variable=self.grid_mode_var,
            command=self.toggle_grid_view
        )
        self.grid_mode_check.pack(side=tk.RIGHT, padx=10)

        # Фрейм для массового добавления комментариев
        self.bulk_comment_frame = ttk.Frame(self.main_container)
        self.bulk_comment_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(self.bulk_comment_frame, text="Массовое добавление комментария:").pack(side=tk.LEFT, padx=5)
        self.bulk_comment_entry = ttk.Entry(self.bulk_comment_frame, width=30)
        self.bulk_comment_entry.pack(side=tk.LEFT, padx=5)
        
        self.add_bulk_comment_btn = ttk.Button(self.bulk_comment_frame, text="Добавить ко всем",
                                             command=self.add_bulk_comment)
        self.add_bulk_comment_btn.pack(side=tk.LEFT, padx=5)
        
        # Фрейм для фотографии и информации
        self.resume_frame = ttk.Frame(self.main_container)
        self.resume_frame.pack(fill=tk.BOTH, expand=True)
        
        # Фото резюме
        self.photo_label = tk.Label(self.resume_frame)
        self.photo_label.pack(pady=10)
        

        # Информация о резюме
        self.info_frame = ttk.Frame(self.resume_frame)
        self.info_frame.pack(fill=tk.X, pady=5)
        
        self.url_label = tk.Label(self.resume_frame, text="", fg="blue", cursor="hand2", font=('Arial', 10))
        self.url_label.pack(pady=(0, 10))
        self.url_label.bind("<Button-1>", self.open_url)
        
        self.info_canvas = tk.Canvas(self.resume_frame, height=180)
        self.info_scroll = ttk.Scrollbar(self.resume_frame, orient="vertical", command=self.info_canvas.yview)
        self.info_inner_frame = ttk.Frame(self.info_canvas)

        self.info_inner_frame.bind("&lt;configure&gt;", lambda e: self.info_canvas.configure(scrollregion=self.info_canvas.bbox("all")))

        self.info_canvas.create_window((0, 0), window=self.info_inner_frame, anchor="nw")
        self.info_canvas.configure(yscrollcommand=self.info_scroll.set)

        self.info_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0,0))
        self.info_scroll.pack(side=tk.RIGHT, fill=tk.Y)

        # Название резюме
        self.name_label = ttk.Label(self.info_inner_frame, text="", font=('Arial', 12, 'bold'))
        self.name_label.pack(anchor=tk.W, pady=(2, 4))

          # Дополнительная информация
        self.details_label = ttk.Label(self.info_inner_frame, text="", justify=tk.LEFT, wraplength=480)
        self.details_label.pack(anchor=tk.W, pady=(0, 6))
        
        # Поле для быстрой оценки
        self.rating_frame = ttk.Frame(self.resume_frame)
        self.rating_frame.pack(pady=10)
        
        ttk.Label(self.rating_frame, text="Оценка (в + цифра):").pack(side=tk.LEFT)
        self.rating_entry = ttk.Entry(self.rating_frame, width=5)
        self.rating_entry.pack(side=tk.LEFT, padx=5)
        self.rating_entry.insert(0, "в")
        self.rating_entry.focus_set()
        
        # Фрейм для навигации
        self.nav_frame = ttk.Frame(self.main_container)
        self.nav_frame.pack(fill=tk.X, pady=10)
        
        # Кнопки навигации
        self.prev_btn = ttk.Button(self.nav_frame, text="◄ Назад", width=15, 
                                 command=self.prev_resume)
        self.prev_btn.pack(side=tk.LEFT, padx=20)
        
        self.counter_label = ttk.Label(self.nav_frame, 
                                     text=f"0 из {len(self.current_display_data)}", 
                                     font=('Arial', 10))
        self.counter_label.pack(side=tk.LEFT, expand=True)
        
        self.next_btn = ttk.Button(self.nav_frame, text="Вперед ►", width=15,
                                 command=self.next_resume)
        self.next_btn.pack(side=tk.RIGHT, padx=20)
        
        # Кнопка для детального просмотра
        self.details_btn = ttk.Button(self.nav_frame, text="Подробнее",
                                    command=self.show_resume_details)
        self.details_btn.pack(side=tk.RIGHT, padx=20)
        
        # Фрейм для сетки фотографий
        self.grid_frame = ttk.Frame(self.main_container)
        self.grid_photo_labels = []
        self.create_grid_view()
        self.grid_frame.pack_forget()
        
        # Фрейм для пагинации
        self.grid_pagination_frame = ttk.Frame(self.main_container)
        self.grid_pagination_frame.pack_forget()
        
        # Элементы пагинации
        self.first_page_btn = ttk.Button(self.grid_pagination_frame, text="<< Первая",
                                       command=lambda: self.go_to_grid_page(0))
        self.first_page_btn.pack(side=tk.LEFT, padx=2)
        
        self.prev_page_btn = ttk.Button(self.grid_pagination_frame, text="< Предыдущая",
                                      command=self.prev_grid_page)
        self.prev_page_btn.pack(side=tk.LEFT, padx=2)
        
        self.page_label = ttk.Label(self.grid_pagination_frame, text="Страница 1 из 1")
        self.page_label.pack(side=tk.LEFT, padx=10)
        
        self.next_page_btn = ttk.Button(self.grid_pagination_frame, text="Следующая >",
                                      command=self.next_grid_page)
        self.next_page_btn.pack(side=tk.LEFT, padx=2)
        
        self.last_page_btn = ttk.Button(self.grid_pagination_frame, text="Последняя >>",
                                      command=lambda: self.go_to_grid_page(self.get_total_grid_pages() - 1))
        self.last_page_btn.pack(side=tk.LEFT, padx=2)
     
    def open_url(self, event):
        """Открывает URL ссылку в браузере"""
        if len(self.current_display_data) == 0:
            return
            
        resume = self.current_display_data.iloc[self.current_index]
        url = resume.get('full_url') or resume.get('short_url')
        
        if pd.notna(url) and url:
            webbrowser.open(url)



    def show_template_dropdown(self):
        """Показывает выпадающий список с шаблонами комментариев"""
        if not self.templates_exist or len(self.templates) == 0:
            messagebox.showinfo("Информация", "Нет доступных шаблонов комментариев")
            return
        
        # Создаем всплывающее меню
        menu = tk.Menu(self.root, tearoff=0)
        
        # Добавляем каждый шаблон в меню
        for _, row in self.templates.iterrows():
            template_text = row['template_text']
            short_text = template_text[:30] + "..." if len(template_text) > 30 else template_text
            menu.add_command(
                label=f"{row['id']}. {short_text}", 
                command=lambda t=template_text: self.insert_template_to_filter(t)
            )
        
        # Показываем меню рядом с кнопкой
        try:
            menu.tk_popup(
                self.template_btn.winfo_rootx(),
                self.template_btn.winfo_rooty() + self.template_btn.winfo_height()
            )
        finally:
            menu.grab_release()
    
    def insert_template_to_filter(self, template_text):
        """Вставляет текст шаблона в поле фильтра"""
        self.filter_entry.delete(0, tk.END)
        self.filter_entry.insert(0, template_text)
        self.filter_entry.focus_set()
    
    def add_template_dialog(self):
        """Открывает диалог добавления нового шаблона"""
        dialog = tk.Toplevel(self.root)
        dialog.title("Добавить новый шаблон")
        dialog.geometry("500x300")
        dialog.resizable(True, True)
        
        # Делаем диалог модальным
        dialog.transient(self.root)
        dialog.grab_set()
        
        # Основной фрейм
        main_frame = ttk.Frame(dialog)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        ttk.Label(main_frame, text="Текст шаблона комментария:").pack(anchor=tk.W, pady=(0, 5))
        
        # Фрейм для текстового поля со скроллом
        text_frame = ttk.Frame(main_frame)
        text_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        # Вертикальный скроллбар
        text_scroll = ttk.Scrollbar(text_frame)
        text_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Текстовое поле
        template_text = tk.Text(text_frame, wrap=tk.WORD, yscrollcommand=text_scroll.set,
                              height=10, width=50)
        template_text.pack(fill=tk.BOTH, expand=True)
        text_scroll.config(command=template_text.yview)
        
        # Фрейм для кнопок внизу
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill=tk.X, pady=(10, 0))
        
        def save_and_close():
            text_content = template_text.get("1.0", tk.END).strip()
            if not text_content:
                messagebox.showwarning("Предупреждение", "Текст шаблона не может быть пустым")
                return
                
            if self.save_template(text_content):
                # Обновляем список шаблонов
                self.templates = self.load_templates()
                dialog.destroy()
        
        # Кнопка Сохранить
        save_btn = ttk.Button(buttons_frame, text="Сохранить", command=save_and_close)
        save_btn.pack(side=tk.LEFT, padx=(0, 10))
        
        # Кнопка Отмена
        cancel_btn = ttk.Button(buttons_frame, text="Отмена", command=dialog.destroy)
        cancel_btn.pack(side=tk.LEFT)
        
        # Пустое пространство для выравнивания
        ttk.Frame(buttons_frame).pack(side=tk.LEFT, expand=True)
        
        # Устанавливаем фокус на текстовое поле
        template_text.focus_set()
        
        # Биндим Enter для сохранения
        dialog.bind('<Return>', lambda e: save_and_close())
        dialog.bind('<Escape>', lambda e: dialog.destroy())

    def add_bulk_comment(self):
        """Добавляет комментарий ко всем отображаемым резюме"""
        comment = self.bulk_comment_entry.get().strip()
        if not comment:
            messagebox.showwarning("Предупреждение", "Введите комментарий для добавления")
            return
            
        if len(self.current_display_data) == 0:
            messagebox.showwarning("Предупреждение", "Нет резюме для добавления комментария")
            return
            
        if not messagebox.askyesno("Подтверждение", 
                                 f"Добавить комментарий '{comment[:30]}...' ко всем {len(self.current_display_data)} отображаемым резюме?"):
            return
            
        try:
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    # Получаем ID всех отображаемых резюме
                    resume_ids = self.current_display_data['id'].tolist()
                    
                    # Обновляем комментарии для всех резюме
                    cursor.executemany("""
                        UPDATE info_res 
                        SET comment = %s 
                        WHERE id = %s
                    """, [(comment, int(resume_id)) for resume_id in resume_ids])
                    
                    conn.commit()
            
            # Обновляем данные в памяти
            self.current_display_data['comment'] = comment
            if not self.filter_mode:
                # Если не в режиме фильтра, обновляем и основную таблицу
                mask = self.resumes['id'].isin(resume_ids)
                self.resumes.loc[mask, 'comment'] = comment
            
            messagebox.showinfo("Успех", f"Комментарий добавлен к {len(resume_ids)} резюме")
            
            # Обновляем отображение
            if self.grid_view_mode:
                self.update_grid_view()
            else:
                self.show_resume()
                
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось добавить комментарий: {e}")



    def create_grid_view(self):
        """Создает сетку 3x3 для просмотра фотографий"""
        for i in range(3):  # строки
            row_frame = ttk.Frame(self.grid_frame)
            row_frame.pack(fill=tk.BOTH, expand=True)
            
            for j in range(3):  # столбцы
                frame = ttk.Frame(row_frame, relief=tk.RIDGE, borderwidth=1)
                frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=2, pady=2)
                
                label = tk.Label(frame)
                label.pack(fill=tk.BOTH, expand=True)
                
                # Добавляем обработчик клика
                label.bind("<Button-1>", lambda e, idx=i*3+j: self.grid_label_click(idx))
                self.grid_photo_labels.append(label)
    
    def get_total_grid_pages(self):
        """Возвращает общее количество страниц в режиме сетки"""
        if len(self.current_display_data) == 0:
            return 1
        return (len(self.current_display_data) // self.grid_page_size + (1 if len(self.current_display_data) % self.grid_page_size > 0 else 0))
    
    def get_current_grid_page(self):
        """Возвращает текущую страницу в режиме сетки"""
        return self.current_index // self.grid_page_size
    
    def go_to_grid_page(self, page):
        """Переходит на указанную страницу в режиме сетки"""
        total_pages = self.get_total_grid_pages()
        if page < 0:
            page = 0
        elif page >= total_pages:
            page = total_pages - 1
        
        self.current_index = page * self.grid_page_size
        self.update_grid_view()
        self.update_pagination_controls()
    
    def prev_grid_page(self):
        """Переход на предыдущую страницу в режиме сетки"""
        self.go_to_grid_page(self.get_current_grid_page() - 1)
    
    def next_grid_page(self):
        """Переход на следующую страницу в режиме сетки"""
        self.go_to_grid_page(self.get_current_grid_page() + 1)
    
    def update_pagination_controls(self):
        """Обновляет элементы управления пагинацией"""
        current_page = self.get_current_grid_page() + 1
        total_pages = self.get_total_grid_pages()
        
        self.page_label.config(text=f"Страница {current_page} из {total_pages}")
        
        # Обновляем состояние кнопок
        self.first_page_btn['state'] = 'normal' if current_page > 1 else 'disabled'
        self.prev_page_btn['state'] = 'normal' if current_page > 1 else 'disabled'
        self.next_page_btn['state'] = 'normal' if current_page < total_pages else 'disabled'
        self.last_page_btn['state'] = 'normal' if current_page < total_pages else 'disabled'
    
    def grid_label_click(self, index):
        """Обрабатывает клик по изображению в сетке"""
        if not self.grid_view_mode:
            return
            
        # Вычисляем реальный индекс резюме
        real_index = self.current_index + index
        
        if real_index < len(self.current_display_data):
            # Переключаем в обычный режим и показываем выбранное резюме
            self.current_index = real_index
            self.grid_mode_var.set(False)
            self.toggle_grid_view()
    
    def toggle_grid_view(self):
        """Переключает между обычным режимом и режимом сетки"""
        self.grid_view_mode = self.grid_mode_var.get()
        
        if self.grid_view_mode:
            # Переключаемся в режим сетки
            self.resume_frame.pack_forget()
            self.nav_frame.pack_forget()
            self.grid_frame.pack(fill=tk.BOTH, expand=True)
            self.grid_pagination_frame.pack(fill=tk.X, pady=5)
            self.update_grid_view()
            self.update_pagination_controls()
        else:
            # Возвращаемся в обычный режим
            self.grid_frame.pack_forget()
            self.grid_pagination_frame.pack_forget()
            self.resume_frame.pack(fill=tk.BOTH, expand=True)
            self.nav_frame.pack(fill=tk.X, pady=10)
            self.show_resume()
    
    def update_grid_view(self):
        """Обновляет сетку фотографий"""
        if not self.grid_view_mode or len(self.current_display_data) == 0:
            return
        
        start_index = self.current_index
        end_index = min(start_index + self.grid_page_size, len(self.current_display_data))
        
        # Очищаем все изображения в сетке
        for label in self.grid_photo_labels:
            label.config(image='', text='')
        
        # Заполняем сетку доступными фотографиями
        for i in range(start_index, end_index):
            resume = self.current_display_data.iloc[i]
            photo_path = self.get_photo_path(resume['html_file_name'], resume['img_text']) if pd.notna(resume['img_text']) else None
            grid_pos = i - start_index
            
            if grid_pos >= len(self.grid_photo_labels):
                break
            
            try:
                if photo_path and os.path.exists(photo_path):
                    cache_key = f"grid_{resume['html_file_name']}_{resume['img_text']}"
                    
                    if cache_key not in self.image_cache:
                        img = Image.open(photo_path)
                        img = img.resize(self.grid_image_size, Image.LANCZOS)
                        self.image_cache[cache_key] = ImageTk.PhotoImage(img)
                    
                    photo_img = self.image_cache[cache_key]
                else:
                    photo_img = self.blank_photo_grid
                
                self.grid_photo_labels[grid_pos].config(image=photo_img)
                self.grid_photo_labels[grid_pos].image = photo_img
                
                # Добавляем подпись с именем и номером
                name = resume['name_resume'] if pd.notna(resume['name_resume']) else "Без названия"
                text = f"{i+1}. {name[:20]}{'...' if len(name) > 20 else ''}"
                self.grid_photo_labels[grid_pos].config(text=text, compound=tk.TOP)
                
            except Exception as e:
                print(f"Ошибка загрузки фото для сетки: {e}")
                self.grid_photo_labels[grid_pos].config(text=f"Ошибка загрузки {i+1}")
        
        # Обновляем счетчик
        self.update_counter()
    
    def apply_filter(self):
        """Применяет фильтр по комментариям, номеру сессии и дате посещения"""
        filter_text = self.filter_entry.get().strip()
        session_from = self.session_from.get().strip()
        session_to = self.session_to.get().strip()
        date_from = self.date_from.get_date()
        date_to = self.date_to.get_date()
        
        try:
            session_from = int(session_from) if session_from else self.session_min
            session_to = int(session_to) if session_to else self.session_max
        except ValueError:
            messagebox.showwarning("Ошибка", "Номера сессий должны быть целыми числами")
            return
        
        self.current_filter = filter_text.lower()
        self.filter_mode = True
        self.current_index = 0
        
        # Фильтруем данные
        filtered_resumes = []
        for _, row in self.resumes.iterrows():
            match = True
            
            # Фильтр по номеру сессии
            if session_from is not None and session_to is not None:
                if not (session_from <= row['session_id'] <= session_to):
                    match = False
            
            # Фильтр по дате посещения
            if match and date_from and date_to:
                visit_date = pd.to_datetime(row['visit_time']).date() if pd.notna(row['visit_time']) else None
                if visit_date is None or not (date_from <= visit_date <= date_to):
                    match = False
            
            # Фильтр по комментарию
            if match and self.current_filter:
                if pd.isna(row['comment']) or self.current_filter not in str(row['comment']).lower():
                    match = False
            
            if match:
                filtered_resumes.append(row)
        
        self.current_display_data = pd.DataFrame(filtered_resumes)
        self.image_cache.clear()
        
        if self.grid_view_mode:
            self.update_grid_view()
        else:
            self.show_resume()
        
        self.update_counter()
    
    def reset_filter(self):
        """Сбрасывает фильтр и показывает все резюме"""
        self.filter_mode = False
        self.current_filter = ""
        self.filter_entry.delete(0, tk.END)
        self.session_from.delete(0, tk.END)
        self.session_from.insert(0, str(self.session_min))
        self.session_to.delete(0, tk.END)
        self.session_to.insert(0, str(self.session_max))
        self.date_from.set_date(self.min_date)
        self.date_to.set_date(self.max_date)
        self.current_index = 0
        self.current_display_data = self.resumes.copy()
        self.image_cache.clear()
        
        if self.grid_view_mode:
            self.update_grid_view()
        else:
            self.show_resume()
        
        self.update_counter()
    
    def show_resume(self):
        """Отображает текущее резюме"""
        if len(self.current_display_data) == 0:
            self.photo_label.config(image='', text='Нет данных для отображения')
            self.name_label.config(text='')
            self.details_label.config(text='')
            self.url_label.config(text='')  # Очищаем ссылку
            # Очищаем поле комментария браузера
            self.browser_comment_entry.delete(0, tk.END)
            return
        
        resume = self.current_display_data.iloc[self.current_index]
        photo_path = self.get_photo_path(resume['html_file_name'], resume['img_text']) if pd.notna(resume['img_text']) else None
        
        try:
            # Загружаем фото
            if photo_path and os.path.exists(photo_path):
                cache_key = f"single_{resume['html_file_name']}_{resume['img_text']}"
                
                if cache_key not in self.image_cache:
                    img = Image.open(photo_path)
                    img = img.resize(self.single_image_size, Image.LANCZOS)
                    self.image_cache[cache_key] = ImageTk.PhotoImage(img)
                
                photo_img = self.image_cache[cache_key]
            else:
                photo_img = self.blank_photo  # Используем белую заглушку
            
            self.photo_label.config(image=photo_img)
            self.photo_label.image = photo_img
            
            # Отображаем кликабельную ссылку
            url = resume.get('full_url') or resume.get('short_url')
            if pd.notna(url) and url:
                # Обрезаем длинные URL для лучшего отображения
                display_url = url
                if len(display_url) > 50:
                    display_url = display_url[:47] + "..."
                self.url_label.config(text=display_url)
            else:
                self.url_label.config(text="")
            
            # Отображаем информацию
            name = resume['name_resume'] if pd.notna(resume['name_resume']) else "Без названия"
            self.name_label.config(text=name)
            
            # Дополнительная информация (убираем URL из details, так как он теперь отдельно)
            details = []
            if pd.notna(resume['session_id']):
                details.append(f"Сессия: {resume['session_id']}")
            if pd.notna(resume['html_file_name']):
                details.append(f"Файл донор: {resume['html_file_name']}")
            if pd.notna(resume['visit_time']):
                details.append(f"Дата: {resume['visit_time']}")
            if pd.notna(resume['comment']):
                details.append(f"Комментарий: {resume['comment']}")
            if pd.notna(resume.get('comment_browser', '')):
                browser_comment = str(resume['comment_browser']).replace('\n', '\n')
                details.append(f"Браузер:\n{browser_comment}")
            
            self.details_label.config(text="\n".join(details))
            
            # Заполняем поле комментария браузера
                # ВСЕГДА ОЧИЩАЕМ поле комментария браузера при показе нового резюме
            self.browser_comment_entry.delete(0, tk.END)
            
            # Устанавливаем фокус на поле оценки
            self.rating_entry.delete(0, tk.END)
            self.rating_entry.insert(0, "в")
            self.rating_entry.focus_set()

        except Exception as e:
            print(f"Ошибка загрузки резюме: {e}")
            self.photo_label.config(image='', text='Ошибка загрузки фото')
            self.name_label.config(text='')
            self.details_label.config(text='')
            self.url_label.config(text='')
            self.browser_comment_entry.delete(0, tk.END)  # Очищаем и при ошибке
    
    def get_photo_path(self, html_file_name, img_text):
        """Возвращает точный путь к фотографии кандидата"""
        if not html_file_name or pd.isna(html_file_name) or not img_text or pd.isna(img_text):
            return None
        
        try:
            # Удаляем расширение .html
            base_name = os.path.splitext(html_file_name)[0]
            photos_dir = os.path.join(self.photos_base_path, f"{base_name}_files")
            
            if not os.path.exists(photos_dir):
                return None
            
            # Получаем чистое имя файла из img_text
            img_filename = os.path.basename(str(img_text).split('?')[0].strip('"\''))
            
            # 1. Попытка найти точное совпадение
            exact_path = os.path.join(photos_dir, img_filename)
            if os.path.exists(exact_path):
                return exact_path
            
            # 2. Если точного совпадения нет, ищем файл с тем же именем, но другим регистром
            for file in os.listdir(photos_dir):
                if file.lower() == img_filename.lower():
                    return os.path.join(photos_dir, file)
            
            return None
            
        except Exception as e:
            print(f"Ошибка поиска фото: {e}")
            return None
    
    def handle_key_press(self, event):
        """Обрабатывает нажатия клавиш"""
        if len(self.current_display_data) == 0:
            return

        focused_widget = self.root.focus_get()

        # Если фокус на поле ввода оценки
        if focused_widget == self.rating_entry:
            if event.char.isdigit():
                self.save_rating(event.char)
                self.next_resume()
            elif event.keysym.startswith('KP_') and event.keysym[3:].isdigit():
                digit = event.keysym[3:]
                self.save_rating(digit)
                self.next_resume()
            return

        # Обработка горячих клавиш
        if event.keysym.lower() == 'r' :  # Клавиша R для вставки из буфера
            self.paste_to_browser_comment_entry()
            return 

        # Если фокус в другом поле ввода, игнорируем навигацию
        if isinstance(focused_widget, (tk.Entry, tk.Text, ttk.Entry, ttk.Combobox)):
            return

        # Обработка навигации
        if not self.grid_view_mode:
            if event.keysym == 'Left':
                self.prev_resume()
            elif event.keysym == 'Right':
                self.next_resume()
            elif event.keysym.lower() == 'g':
                self.grid_mode_var.set(not self.grid_mode_var.get())
                self.toggle_grid_view()
        else:
            if event.keysym == 'Left':
                self.prev_grid_page()
            elif event.keysym == 'Right':
                self.next_grid_page()
            elif event.keysym == 'Up':
                self.prev_grid_page()
            elif event.keysym == 'Down':
                self.next_grid_page()
            elif event.char.isdigit() and 1 <= int(event.char) <= 9:
                self.select_from_grid(int(event.char) - 1)
            elif event.keysym == 'Escape':
                self.grid_mode_var.set(False)
                self.toggle_grid_view()
    
    def paste_from_clipboard(self):
        """Вставляет текст из буфера обмена в comment_browser"""
        if len(self.current_display_data) == 0:
            return
        
        try:
            # Получаем текст из буфера обмена
            clipboard_text = pyperclip.paste().strip()
            if not clipboard_text:
                messagebox.showinfo("Информация", "Буфер обмена пуст")
                return
            
            # Сохраняем в базу данных
            resume = self.current_display_data.iloc[self.current_index]
            
            if not self.comment_browser_exists:
                # Если колонки нет, создаем ее
                self.create_comment_browser_column()
            
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        UPDATE info_res 
                        SET comment_browser = %s 
                        WHERE id = %s
                    """, (clipboard_text, int(resume['id'])))
                    conn.commit()
            
            # Обновляем данные в памяти
            self.current_display_data.at[self.current_index, 'comment_browser'] = clipboard_text
            if not self.filter_mode:
                self.resumes.loc[self.resumes['id'] == resume['id'], 'comment_browser'] = clipboard_text
            
            messagebox.showinfo("Успех", "Текст из буфера обмена сохранен в comment_browser")
            
            # Обновляем отображение
            if self.grid_view_mode:
                self.update_grid_view()
            else:
                self.show_resume()
                
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось сохранить текст из буфера: {e}")
    
    def save_browser_comment(self):
        """Сохраняет комментарий из браузера с накоплением"""
        if len(self.current_display_data) == 0:
            messagebox.showwarning("Предупреждение", "Нет резюме для сохранения комментария")
            return

        new_browser_comment = self.browser_comment_entry.get().strip()
        if not new_browser_comment:
            messagebox.showwarning("Предупреждение", "Введите комментарий для сохранения")
            return

        resume = self.current_display_data.iloc[self.current_index]

        if not self.comment_browser_exists:
            # Если колонки нет, создаем ее
            self.create_comment_browser_column()

        try:
            # Получаем текущий комментарий из базы
            current_comment = ""
            if pd.notna(resume.get('comment_browser', '')):
                current_comment = str(resume['comment_browser']).strip()

            # Формируем обновленный комментарий с ПЕРЕНОСАМИ СТРОК
            if current_comment:
                # Если уже есть комментарии, добавляем новый с ПЕРЕНОСОМ СТРОКИ
                updated_comment = f"{current_comment}\n{new_browser_comment}"
            else:
                updated_comment = new_browser_comment

            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        UPDATE info_res 
                        SET comment_browser = %s 
                        WHERE id = %s
                    """, (updated_comment, int(resume['id'])))
                    conn.commit()

            # Обновляем данные в памяти
            self.current_display_data.at[self.current_index, 'comment_browser'] = updated_comment
            if not self.filter_mode:
                self.resumes.loc[self.resumes['id'] == resume['id'], 'comment_browser'] = updated_comment

            # ОЧИСТКА ПОЛЯ ВВОДА ПОСЛЕ УСПЕШНОГО СОХРАНЕНИЯ
            self.browser_comment_entry.delete(0, tk.END)

            messagebox.showinfo("Успех", "Комментарий из браузера добавлен")

            # Обновляем отображение
            if self.grid_view_mode:
                self.update_grid_view()
            else:
                self.show_resume()

        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось сохранить комментарий: {e}")
    
    def create_comment_browser_column(self):
        """Создает колонку comment_browser если она не существует"""
        try:
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        ALTER TABLE info_res 
                        ADD COLUMN IF NOT EXISTS comment_browser TEXT
                    """)
                    conn.commit()
                    self.comment_browser_exists = True
                    print("Колонка comment_browser создана")
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось создать колонку comment_browser: {e}")
    
    def select_from_grid(self, position):
        """Выбирает резюме из сетки по позиции"""
        if position < 0 or position >= 9:
            return
        
        actual_index = self.current_index + position
        if actual_index >= len(self.current_display_data):
            return
        
        # Переключаемся в обычный режим и показываем выбранное резюме
        self.current_index = actual_index
        self.grid_mode_var.set(False)
        self.toggle_grid_view()
    
    def update_counter(self):
        """Обновляет счетчик просмотренных резюме"""
        total = len(self.current_display_data)
        if total == 0:
            self.counter_label.config(text="0 из 0")
        else:
            if self.grid_view_mode:
                # В режиме сетки показываем диапазон
                start = self.current_index + 1
                end = min(self.current_index + self.grid_page_size, total)
                self.counter_label.config(text=f"{start}-{end} из {total}")
            else:
                self.counter_label.config(text=f"{self.current_index + 1} из {total}")
    
    def prev_page(self):
        """Переход на предыдущую страницу в режиме сетки"""
        if self.current_index >= 9:
            self.current_index -= 9
            self.update_grid_view()
            self.update_counter()
    
    def next_page(self):
        """Переход на следующую страницу в режиме сетки"""
        if self.current_index + 9 < len(self.current_display_data):
            self.current_index += 9
            self.update_grid_view()
            self.update_counter()
    
    def save_rating(self, digit):
        """Сохраняет оценку (в + цифра)"""
        if len(self.current_display_data) == 0:
            return
        
        resume = self.current_display_data.iloc[self.current_index]
        comment = str(f"в{digit}")
        
        try:
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    cursor.execute("""
                        UPDATE info_res 
                        SET comment = %s 
                        WHERE id = %s
                    """, (comment, int(resume['id'])))
                    conn.commit()
            
            # Обновляем данные в памяти
            self.current_display_data.at[self.current_index, 'comment'] = comment
            if not self.filter_mode:
                self.resumes.loc[self.resumes['id'] == resume['id'], 'comment'] = comment
            
            # Обновляем отображение
            if self.grid_view_mode:
                self.update_grid_view()
            else:
                self.show_resume()
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось сохранить оценку: {e}")
    
    def prev_resume(self):
        """Переход к предыдущему резюме"""
        if len(self.current_display_data) == 0:
            return
        
        if self.current_index > 0:
            self.current_index -= 1
            if self.grid_view_mode:
                self.update_grid_view()
            else:
                self.show_resume()
            self.update_counter()
    
    def next_resume(self):
        """Переход к следующему резюме"""
        if len(self.current_display_data) == 0:
            return
        
        if self.current_index < len(self.current_display_data) - 1:
            self.current_index += 1
            if self.grid_view_mode:
                self.update_grid_view()
            else:
                self.show_resume()
            self.update_counter()
    
    def update_counter(self):
        """Обновляет счетчик просмотренных резюме"""
        total = len(self.current_display_data)
        if total == 0:
            self.counter_label.config(text="0 из 0")
        else:
            if self.grid_view_mode:
                # В режиме сетки показываем диапазон
                start = self.current_index + 1
                end = min(self.current_index + 9, total)
                self.counter_label.config(text=f"{start}-{end} из {total}")
            else:
                self.counter_label.config(text=f"{self.current_index + 1} из {total}")
    
    def show_resume_details(self):
        """Показывает детальную информацию о текущем резюме"""
        if len(self.current_display_data) == 0:
            return
        
        resume = self.current_display_data.iloc[self.current_index].to_dict()
        
        detail_window = tk.Toplevel(self.root)
        detail_window.title(f"Резюме: {resume.get('name_resume', 'Без названия')}")
        detail_window.geometry("1200x800")

        main_frame = ttk.Frame(detail_window)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Фрейм для фотографии
        photo_frame = ttk.Frame(main_frame)
        photo_frame.pack(fill=tk.X, pady=10)

        try:
            photo_path = self.get_photo_path(resume.get('html_file_name'), resume.get('img_text')) if pd.notna(resume.get('img_text')) else None
            if photo_path and os.path.exists(photo_path):
                img = Image.open(photo_path)
                img.thumbnail((300, 300))
                photo_img = ImageTk.PhotoImage(img)
            else:
                if self.blank_photo:
                    photo_img = self.blank_photo
                else:
                    img = Image.new('RGB', (300, 300), 'white')
                    photo_img = ImageTk.PhotoImage(img)

            photo_label = tk.Label(photo_frame, image=photo_img)
            photo_label.image = photo_img
            photo_label.pack()
        except Exception as e:
            print(f"Ошибка загрузки фото: {e}")
    
        # Фрейм для информации
        info_frame = ttk.Frame(main_frame)
        info_frame.pack(fill=tk.BOTH, expand=True)
        
        # Создаем Notebook для вкладок
        notebook = ttk.Notebook(info_frame)
        notebook.pack(fill=tk.BOTH, expand=True)
        
        # Вкладка с основной информацией
        general_tab = ttk.Frame(notebook)
        notebook.add(general_tab, text="Основная информация")
        
        fields = [
            ("Название резюме:", resume['name_resume']),
            ("Номер сессии:", resume['session_id']),
            ("Дата посещения:", resume['visit_time']),
            ("URL:", resume['full_url'],)
        ]
        
        for i, (label, value) in enumerate(fields):
            ttk.Label(general_tab, text=label, font=('Arial', 10, 'bold')).grid(
                row=i, column=0, sticky=tk.W, padx=5, pady=2)
            
            if label == "URL:":
                url_label = ttk.Label(general_tab, text=value, foreground="blue", cursor="hand2")
                url_label.grid(row=i, column=1, sticky=tk.W, padx=5, pady=2)
                url_label.bind("<Button-1>", lambda e, url=value: webbrowser.open(url))
            else:
                ttk.Label(general_tab, text=value, wraplength=400).grid(
                    row=i, column=1, sticky=tk.W, padx=5, pady=2)
        
        # Вкладка с комментариями
        comments_tab = ttk.Frame(notebook)
        notebook.add(comments_tab, text="Комментарии")
        
        # Комментарий к резюме
        ttk.Label(comments_tab, text="Комментарий к резюме:", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=5)
        
        self.resume_comment_text = tk.Text(comments_tab, height=5, width=60, wrap=tk.WORD)
        self.resume_comment_text.pack(fill=tk.X, pady=5)
        self.resume_comment_text.insert(tk.END, resume.get('comment', '') if pd.notna(resume.get('comment')) else "")

        
        # Комментарий к сессии
        ttk.Label(comments_tab, text="Комментарий к сессии:", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=5)
        
        self.session_comment_text = tk.Text(comments_tab, height=5, width=60, wrap=tk.WORD)
        self.session_comment_text.pack(fill=tk.X, pady=5)
        self.session_comment_text.insert(tk.END, resume.get('session_comment', '') if pd.notna(resume.get('session_comment')) else "")
        
        # Фрейм для кнопок
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill=tk.X, pady=10)
        
        # Кнопка сохранения комментариев
        save_btn = ttk.Button(buttons_frame, text="Сохранить все комментарии", 
                             command=lambda: self.save_all_comments(resume['id'], resume['session_id'], detail_window))
        save_btn.pack(side=tk.LEFT, padx=5)
        
        # Кнопка закрытия
        close_btn = ttk.Button(buttons_frame, text="Закрыть", command=detail_window.destroy)
        close_btn.pack(side=tk.RIGHT, padx=5)
    
    def paste_to_browser_comment_entry(self):
        """Вставляет текст из буфера обмена между комментарием и датой"""
        if len(self.current_display_data) == 0:
            return
    
        try:
            # Удаляем символ 'r', который уже успел ввестись
            current_text = self.browser_comment_entry.get()
            if current_text.endswith('r'):
                self.browser_comment_entry.delete(len(current_text)-1, tk.END)
            
            clipboard_text = pyperclip.paste().strip()
            if not clipboard_text:
                messagebox.showinfo("Информация", "Буфер обмена пуст")
                return
    
            # Получаем текущую дату
            current_date = datetime.now().strftime("%d.%m.%Y")
    
            # Формируем новый текст для добавления (с переносом строки)
            new_text = f"[{clipboard_text}]({current_date})"
    
            # Вставляем в поле
            self.browser_comment_entry.insert(tk.END, new_text)
    
            # Устанавливаем фокус на поле
            self.browser_comment_entry.focus_set()
    
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось вставить из буфера: {e}")
    
    def save_all_comments(self, resume_id, session_id, detail_window=None):
        """Сохраняет все комментарии (включая comment_browser)"""
        resume_comment = str(self.resume_comment_text.get("1.0", tk.END)).strip()
        session_comment = str(self.session_comment_text.get("1.0", tk.END)).strip()
        
        try:
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    # Обновляем комментарий к резюме
                    cursor.execute("""
                        UPDATE info_res 
                        SET comment = %s
                        WHERE id = %s
                    """, (resume_comment, resume_id))
                    
                    # Обновляем комментарий к сессии
                    cursor.execute("""
                        UPDATE parsing_metadata 
                        SET parsing_comment = %s 
                        WHERE session_id = %s
                    """, (session_comment, session_id))
                    
                    conn.commit()
            
            messagebox.showinfo("Успех", "Все комментарии успешно сохранены")
            self.load_data()
            
            if self.filter_mode:
                self.apply_filter()
            else:
                self.current_display_data = self.resumes.copy()
            
            if self.grid_view_mode:
                self.update_grid_view()
            else:
                self.show_resume()
            
            if detail_window:
                detail_window.destroy()
                
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось сохранить комментарии: {e}")
    
    def clear_comments(self, resume_id, session_id, detail_window=None):
        """Удаляет комментарии к резюме и сессии"""
        if messagebox.askyesno("Подтверждение", "Вы уверены, что хотите удалить все комментарии?"):
            try:
                with psycopg2.connect(**self.db_config) as conn:
                    with conn.cursor() as cursor:
                        # Удаляем комментарий к резюме
                        cursor.execute("""
                            UPDATE info_res 
                            SET comment = %s 
                            WHERE id = %s
                        """, ('' , int(resume_id),))
                        
                        # Удаляем комментарий к сессии
                        cursor.execute("""
                            UPDATE parsing_metadata 
                            SET parsing_comment = %s 
                            WHERE session_id = %s
                        """, ('', session_id,))
                        
                        conn.commit()
                
                self.resume_comment_text.delete("1.0", tk.END)
                self.session_comment_text.delete("1.0", tk.END)
                messagebox.showinfo("Успех", "Комментарии удалены")
                self.load_data()

                if self.filter_mode:
                    self.apply_filter()  # Переприменяем фильтр, если он активен
                else:
                    self.current_display_data = self.resumes.copy()
                
                if self.grid_view_mode:
                    self.update_grid_view()
                else:
                    self.show_resume()
                
                if detail_window:
                    detail_window.destroy()
                    
            except Exception as e:
                messagebox.showerror("Ошибка", f"Не удалось сохранить комментарии: {e}")


if __name__ == "__main__":
    root = tk.Tk()
    app = ResumeViewerApp(root)
    root.mainloop()


C:\Users\sekin\AppData\Local\Temp\ipykernel_25064\1706532804.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  self.resumes = pd.read_sql(query, conn)
C:\Users\sekin\AppData\Local\Temp\ipykernel_25064\1706532804.py:159: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  session_range = pd.read_sql("""
C:\Users\sekin\AppData\Local\Temp\ipykernel_25064\1706532804.py:167: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  date_range = pd.read_sql("""


Ошибка загрузки резюме: 'ResumeViewerApp' object has no attribute 'blank_photo'


KeyError: 'html_file_name'

In [20]:
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',
    'user': 'postgres',
    'password': '123'
}

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cursor:
        cursor.execute('''ALTER TABLE info_res
ADD COLUMN comment_browser TEXT;''')
        conn.commit()
                            